# Generate training data

In [37]:
import pandas as pd
import re
from pathlib import Path
from typing import Dict, List, Tuple, Set
import warnings
warnings.filterwarnings('ignore')

class ControlMappingTrainingSetGenerator:
    """
    Genera un training set unificato.
    
    MODIFICHE ATTUALI:
    - Aggregazione SecNumCloud: Se un controllo EUCS mappa su una sezione (es. 6.1),
      tutti i sottocontrolli (6.1.a, 6.1.b) vengono concatenati in un UNICO target.
    - Utilizzo Description_EN per SecNumCloud.
    - Esclusione BSI C5.
    """
    
    def __init__(self, base_path: str = "./Schemes"):
        self.base_path = Path(base_path)
        self.data = {}
        self.training_rows = []
        
    def clean_control_id(self, control_id: str) -> str:
        """Pulisce e normalizza gli ID dei controlli"""
        if pd.isna(control_id) or control_id == '':
            return ''
        
        control_id = str(control_id).strip()
        control_id = re.sub(r'^(EUCS\s+|ISO\s+)', '', control_id, flags=re.IGNORECASE)
        return control_id
    
    def parse_control_list(self, control_string: str) -> List[str]:
        """Parsifica stringhe con multipli ID"""
        if pd.isna(control_string) or control_string == '':
            return []
        
        controls = re.split(r'[,\n]+', str(control_string))
        cleaned = []
        for ctrl in controls:
            ctrl = ctrl.strip()
            if ctrl:
                cleaned.append(self.clean_control_id(ctrl))
        return cleaned

    def find_secnumcloud_matches(self, source_ref: str, lookup_dict: Dict) -> List[Dict]:
        """
        Trova tutti i controlli SecNumCloud che corrispondono (esatto o per prefisso).
        Es: '6.1' -> ['6.1.a', '6.1.b']
        """
        matches = []
        source_ref = self.clean_control_id(source_ref)
        if not source_ref:
            return matches

        # 1. Match Esatto
        if source_ref in lookup_dict:
            matches.append(lookup_dict[source_ref])
            return matches

        # 2. Match Gerarchico (Prefix)
        prefix = source_ref + "."
        matching_keys = [k for k in lookup_dict.keys() if k.startswith(prefix)]
        
        # Ordina per mantenere coerenza (es: 6.1.a prima di 6.1.b)
        matching_keys.sort()
        
        for key in matching_keys:
            matches.append(lookup_dict[key])
            
        return matches
    
    def load_cisco(self) -> pd.DataFrame:
        print("📂 Caricamento Cisco.csv...")
        df = pd.read_csv(self.base_path / "Cisco.csv")
        return df
    
    def load_spanish_ens(self) -> pd.DataFrame:
        print("📂 Caricamento SpanishENS.csv...")
        df = pd.read_csv(self.base_path / "SpanishENS.csv")
        return df
    
    def load_secnumcloud(self) -> pd.DataFrame:
        print("📂 Caricamento SecNumCloud...")
        df = pd.read_csv(self.base_path / "secnumcloud_controlsParsedAndTranslatedEnglish.csv")
        return df
    
    def load_eucs_medina(self) -> pd.DataFrame:
        print("📂 Caricamento EUCS MEDINA...")
        df = pd.read_csv(self.base_path / "NewEucsRequirements.csv")
        return df
    
    def load_fabasoft_metrics(self) -> pd.DataFrame:
        print("📂 Caricamento Fabasoft Metrics...")
        df = pd.read_csv(self.base_path / "fabasoftMetrics.csv")
        return df
    
    def load_all_data(self):
        print("\n🔄 CARICAMENTO DATI\n" + "="*60)
        self.data['cisco'] = self.load_cisco()
        self.data['spanish_ens'] = self.load_spanish_ens()
        self.data['secnumcloud'] = self.load_secnumcloud()
        self.data['eucs_medina'] = self.load_eucs_medina()
        self.data['fabasoft'] = self.load_fabasoft_metrics()
        print("\n✅ Tutti i file caricati con successo!\n")
    
    def create_lookup_dict(self, df: pd.DataFrame, id_col: str, text_cols: List[str], 
                           scheme_name: str) -> Dict[str, Dict]:
        lookup = {}
        for _, row in df.iterrows():
            control_id = self.clean_control_id(row[id_col])
            if not control_id:
                continue
            
            text_parts = []
            for col in text_cols:
                if col in df.columns and pd.notna(row[col]):
                    text_parts.append(str(row[col]))
            
            text = ' '.join(text_parts).strip()
            
            lookup[control_id] = {
                'id': control_id,
                'text': text,
                'scheme': scheme_name,
                'original_id': row[id_col]
            }
        return lookup
    
    def process_cisco_mappings(self):
        print("\n🔗 PROCESSING CISCO MAPPINGS\n" + "="*60)
        cisco_df = self.data['cisco']
        
        spanish_lookup = self.create_lookup_dict(self.data['spanish_ens'], 'Control ID', ['Description'], 'SpanishENS')
        secnum_lookup = self.create_lookup_dict(self.data['secnumcloud'], 'ID', ['Description_EN'], 'SecNumCloud')
        eucs_lookup = self.create_lookup_dict(self.data['eucs_medina'], 'Code', ['EUCS Control (2022)', 'EUCS Text'], 'EUCS')
        
        count = 0
        for idx, row in cisco_df.iterrows():
            source_id = row['Control Reference']
            source_text = f"{row['Control Title']} - {row['Control Wording']}"
            mapped_controls = []
            
            # Spanish ENS (Standard mapping)
            for col in ['Spanish ENS BASIC Control', 'Spanish ENS Medium Control', 'Spanish ENS High Control']:
                if col in cisco_df.columns and pd.notna(row[col]):
                    for ctrl_id in self.parse_control_list(row[col]):
                        if ctrl_id in spanish_lookup:
                            mapped_controls.append(spanish_lookup[ctrl_id])
            
            # EUCS (Standard mapping)
            if 'EUCS Basic Control' in cisco_df.columns and pd.notna(row['EUCS Basic Control']):
                for ctrl_id in self.parse_control_list(row['EUCS Basic Control']):
                    if ctrl_id in eucs_lookup:
                        mapped_controls.append(eucs_lookup[ctrl_id])
            
            # SecNumCloud (AGGREGATO)
            if 'SecNumCloud Control' in cisco_df.columns and pd.notna(row['SecNumCloud Control']):
                all_secnum_matches = []
                control_ids = self.parse_control_list(row['SecNumCloud Control'])
                
                # Raccogli TUTTI i match
                for ctrl_id in control_ids:
                    found = self.find_secnumcloud_matches(ctrl_id, secnum_lookup)
                    all_secnum_matches.extend(found)
                
                # Se abbiamo trovato match, aggreghiamoli in UN solo oggetto target
                if all_secnum_matches:
                    # Rimuovi duplicati basati su ID
                    unique_matches = {m['id']: m for m in all_secnum_matches}.values()
                    
                    # Concatena ID e Testi
                    combined_id = " | ".join([m['id'] for m in unique_matches])
                    combined_text = " ".join([m['text'] for m in unique_matches])
                    
                    mapped_controls.append({
                        'id': combined_id,
                        'text': combined_text,
                        'scheme': 'SecNumCloud',
                        'original_id': row['SecNumCloud Control']
                    })

            if mapped_controls:
                self.training_rows.append({
                    'source_scheme': 'Cisco',
                    'source_id': source_id,
                    'source_text': source_text,
                    'mapped_controls': mapped_controls
                })
                count += 1
                
        print(f"✓ Cisco processato: {count} righe generate")

    def process_eucs_medina_mappings(self):
        print("\n🔗 PROCESSING EUCS MEDINA MAPPINGS\n" + "="*60)
        eucs_df = self.data['eucs_medina']
        
        secnum_lookup = self.create_lookup_dict(self.data['secnumcloud'], 'ID', ['Description_EN'], 'SecNumCloud')
        
        count = 0
        
        for idx, row in eucs_df.iterrows():
            source_id = row['Code'] if pd.notna(row['Code']) else f"EUCS-{idx}"
            source_text = f"{row['EUCS Control (2022)']} - {row['EUCS Text']}"
            
            mapped_controls = []
            
            # Mapping verso SecNumCloud con AGGREGAZIONE
            if 'SecNumCloud FRANCE ' in eucs_df.columns and pd.notna(row['SecNumCloud FRANCE ']):
                all_secnum_matches = []
                control_ids = self.parse_control_list(row['SecNumCloud FRANCE '])
                
                # 1. Trova tutti i sottocontrolli (es. 6.1 -> 6.1.a, 6.1.b...)
                for ctrl_id in control_ids:
                    found = self.find_secnumcloud_matches(ctrl_id, secnum_lookup)
                    all_secnum_matches.extend(found)
                
                # 2. Aggrega in una singola entry
                if all_secnum_matches:
                    # Deduping (caso raro ma possibile)
                    unique_matches = {m['id']: m for m in all_secnum_matches}.values()
                    
                    # Concatena gli ID (per riferimento) e i Testi (per embedding)
                    # Esempio ID: "6.1.a | 6.1.b"
                    combined_id = " | ".join([m['id'] for m in unique_matches])
                    combined_text = " ".join([m['text'] for m in unique_matches])
                    
                    mapped_controls.append({
                        'id': combined_id,
                        'text': combined_text,
                        'scheme': 'SecNumCloud',
                        'original_id': row['SecNumCloud FRANCE ']
                    })

            if mapped_controls:
                self.training_rows.append({
                    'source_scheme': 'EUCS',
                    'source_id': source_id,
                    'source_text': source_text,
                    'mapped_controls': mapped_controls
                })
                count += 1
        
        print(f"✓ EUCS MEDINA processato: {count} righe generate")
    
    def process_fabasoft_mappings(self):
        print("\n🔗 PROCESSING FABASOFT METRICS MAPPINGS\n" + "="*60)
        fabasoft_df = self.data['fabasoft']
        eucs_lookup = self.create_lookup_dict(self.data['eucs_medina'], 'Code', ['EUCS Control (2022)', 'EUCS Text'], 'EUCS')
        
        count = 0
        for idx, row in fabasoft_df.iterrows():
            source_id = row['ID'] if pd.notna(row['ID']) else f"Metric-{idx}"
            source_text = f"{row['Name']} - {row['Description']}" if pd.notna(row['Description']) else row['Name']
            
            mapped_controls = []
            
            if 'Possible Control ID\nScheme' in fabasoft_df.columns and pd.notna(row['Possible Control ID\nScheme']):
                lines = str(row['Possible Control ID\nScheme']).split('\n')
                for line in lines:
                    line = line.strip()
                    if 'EUCS' in line:
                        match = re.search(r'EUCS\s+([A-Z]+-\d+)', line)
                        if match:
                            ctrl_id = self.clean_control_id(match.group(1))
                            if ctrl_id in eucs_lookup:
                                mapped_controls.append(eucs_lookup[ctrl_id])
            
            if mapped_controls:
                self.training_rows.append({
                    'source_scheme': 'Fabasoft',
                    'source_id': source_id,
                    'source_text': source_text,
                    'mapped_controls': mapped_controls
                })
                count += 1
                
        print(f"✓ Fabasoft processato: {count} righe generate")
    
    def create_training_dataframe(self) -> pd.DataFrame:
        print("\n📊 CREAZIONE TRAINING DATAFRAME\n" + "="*60)
        records = []
        for row in self.training_rows:
            source_scheme = row['source_scheme']
            source_id = row['source_id']
            source_text = row['source_text']
            
            for mapped in row['mapped_controls']:
                records.append({
                    'source_scheme': source_scheme,
                    'source_id': source_id,
                    'source_text': source_text,
                    'target_scheme': mapped['scheme'],
                    'target_id': mapped['id'], # Qui ora avremo ID multipli es "6.1.a | 6.1.b"
                    'target_text': mapped['text'], # E testo concatenato
                    'original_target_id': mapped['original_id']
                })
        
        df = pd.DataFrame(records)
        print(f"✓ Training set creato: {len(df)} righe")
        return df
    
    def validate_mappings(self, df: pd.DataFrame):
        print("\n🔍 VALIDAZIONE MAPPINGS\n" + "="*60)
        if df.empty:
            print("  ⚠️ DataFrame vuoto.")
            return

        print("\n1. Controllo Aggregazione SecNumCloud (Top 5 righe):")
        # Filtra solo le righe dove il target è SecNumCloud
        sn_rows = df[df['target_scheme'] == 'SecNumCloud']
        if not sn_rows.empty:
            for _, row in sn_rows.head(5).iterrows():
                print(f"\n   Source: [{row['source_scheme']}] {row['source_id']}")
                print(f"   Target ID Composite: {row['target_id']}")
                # Mostra un'anteprima del testo per verificare la concatenazione
                preview_text = row['target_text'][:100] + "..." if len(row['target_text']) > 100 else row['target_text']
                print(f"   Target Text Start: {preview_text}")
        else:
            print("   Nessun mapping verso SecNumCloud trovato.")

    def save_training_set(self, df: pd.DataFrame, output_path: str = "training_set_control_mappings.csv"):
        df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"\n💾 Training set salvato in: {output_path}")

    def generate(self, output_path: str = "training_set_control_mappings.csv"):
        self.load_all_data()
        self.process_cisco_mappings()
        self.process_eucs_medina_mappings()
        self.process_fabasoft_mappings()
        df = self.create_training_dataframe()
        self.validate_mappings(df)
        self.save_training_set(df, output_path)
        return df

if __name__ == "__main__":
    generator = ControlMappingTrainingSetGenerator(base_path="./Schemes")
    training_df = generator.generate(output_path="TrainAndTestData/trainData/training_set_control_mappings.csv")


🔄 CARICAMENTO DATI
📂 Caricamento Cisco.csv...
📂 Caricamento SpanishENS.csv...
📂 Caricamento SecNumCloud...
📂 Caricamento EUCS MEDINA...
📂 Caricamento Fabasoft Metrics...

✅ Tutti i file caricati con successo!


🔗 PROCESSING CISCO MAPPINGS
✓ Cisco processato: 181 righe generate

🔗 PROCESSING EUCS MEDINA MAPPINGS
✓ EUCS MEDINA processato: 78 righe generate

🔗 PROCESSING FABASOFT METRICS MAPPINGS
✓ Fabasoft processato: 11 righe generate

📊 CREAZIONE TRAINING DATAFRAME
✓ Training set creato: 278 righe

🔍 VALIDAZIONE MAPPINGS

1. Controllo Aggregazione SecNumCloud (Top 5 righe):

   Source: [Cisco] CCF 1
   Target ID Composite: 18.2.1.a | 18.3.a | 18.4.a
   Target Text Start: The service provider must document and implement an audit program over three years defining the peri...

   Source: [Cisco] CCF 2
   Target ID Composite: 12.1.a
   Target Text Start: The service provider must document the operating procedures, keep them up to date and make them acce...

   Source: [Cisco] CCF 4
   Tar